In [ ]:
# Create SVG figures and CSVs for the five experiments,
# plus four simple conceptual SVG schematics for undergrads.
# Everything is saved under /mnt/data so you can download/use directly.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.fft import fft2, ifft2, fftfreq
from pathlib import Path

# Output dirs
PROJECT_ROOT_NAME = "X-theta-framework"
repo_root = Path.cwd().resolve()
for p in [repo_root, *repo_root.parents]:
    if p.name == PROJECT_ROOT_NAME:
        repo_root = p; break
BASE = repo_root / "paper"


FIGS = BASE / "figs"
DATA = BASE / "data"
FIGS.mkdir(exist_ok=True, parents=True)
DATA.mkdir(exist_ok=True, parents=True)

# --- helpers ---
def save_csv(df, name):
    p = DATA / name
    df.to_csv(p, index=False)
    print(f"[CSV] {p}")
    return p

def save_fig_svg(name):
    p = FIGS / name
    plt.tight_layout()
    plt.savefig(p, dpi=150, bbox_inches="tight", format="svg")
    print(f"[SVG] {p}")
    plt.close()
    return p

# Physical constants (SI)
hbar = 1.054_571_817e-34  # J·s
G    = 6.674_30e-11       # m^3 kg^-1 s^-2
pi   = np.pi

# ==========================================================
# EXPERIMENT 1: θ–AB fringes vs phi_theta
# ==========================================================
phi_min, phi_max = -3*np.pi, 3*np.pi
Nphi             = 400
visibility       = 0.92
phase_bias       = 0.25*np.pi

phi = np.linspace(phi_min, phi_max, Nphi)
intensity = 0.5*(1 + visibility*np.cos(phi + phase_bias))

plt.figure()
plt.plot(phi, intensity)
plt.xlabel(r"$\phi_\theta$")
plt.ylabel("Interference intensity (arb.)")
plt.title("θ–AB fringes vs. effective flux $\phi_\theta$ (period $2\pi$)")
f1 = save_fig_svg("exp1_fringe_vs_phi.svg")

df1 = pd.DataFrame({"phi_theta": phi, "intensity": intensity})
c1 = save_csv(df1, "exp1_fringe_vs_phi.csv")


# ==========================================================
# EXPERIMENT 2: Cross–Hall drift via split-step
# ==========================================================
# knobs
m      = 1.44e-25        # kg
I      = 1.0e-38         # J·s^2
qtheta = 1.0e-34
ell    = 1
A0     = 0.0
gradA  = 5.0e-6
Lx     = 2.0e-3
Ly     = 2.0e-3
Nx     = 128
Ny     = 128

x = np.linspace(-Lx/2, Lx/2, Nx, endpoint=False)
y = np.linspace(-Ly/2, Ly/2, Ny, endpoint=False)
dx = x[1]-x[0]
dy = y[1]-y[0]
X, Y = np.meshgrid(x, y, indexing='xy')

kx = 2*np.pi*fftfreq(Nx, d=dx)
ky = 2*np.pi*fftfreq(Ny, d=dy)
KX, KY = np.meshgrid(kx, ky, indexing='xy')

A_theta = A0 + gradA*Y
V = -(hbar*ell*qtheta/I) * A_theta + (qtheta**2/(2*I)) * (A_theta**2)

x0, y0 = -0.6*Lx, 0.0
sigma  = 0.12e-3
p0x    = 3.0e-27
psi0   = np.exp(-((X-x0)**2 + (Y-y0)**2)/(2*sigma**2)) * np.exp(1j * (p0x/hbar) * X)
psi0   = psi0/np.sqrt((np.abs(psi0)**2).sum())

T_list   = np.array([0.0020, 0.0025, 0.0030, 0.0035])  # seconds
Nt       = 240

def propagate(psi, T, Nt):
    dt = T/Nt
    Kdt = np.exp(-1j * (hbar/(2*m)) * (KX**2 + KY**2) * dt)
    psi_t = psi.copy()
    y_centroids = []
    for _ in range(Nt):
        psi_t *= np.exp(-1j * V * (dt/2) / hbar)
        psi_k = fft2(psi_t)
        psi_k *= Kdt
        psi_t = ifft2(psi_k)
        psi_t *= np.exp(-1j * V * (dt/2) / hbar)
        prob = np.abs(psi_t)**2
        prob /= prob.sum()
        y_centroids.append((prob*Y).sum())
    return np.array(y_centroids), psi_t

# demo trace
T_demo = T_list[1]
y_traj, psi_f = propagate(psi0, T_demo, Nt)

plt.figure()
plt.plot(np.linspace(0, T_demo, len(y_traj)), y_traj)
plt.xlabel("t [s]")
plt.ylabel(r"$\langle y(t)\rangle$ [m]")
plt.title("Transverse centroid vs. time (single run)")
f2a = save_fig_svg("exp2_y_traj.svg")

rows = []
for T in T_list:
    yt, _ = propagate(psi0, T, Nt)
    dy = yt[-1] - yt[0]
    rows.append((T, T**2, dy))
df2 = pd.DataFrame(rows, columns=["T_s", "T2_s2", "delta_y_m"])
c2 = save_csv(df2, "exp2_drift_T2.csv")

# plot dy vs T^2 with fit
plt.figure()
plt.scatter(df2["T2_s2"], df2["delta_y_m"])
coef = np.polyfit(df2["T2_s2"].values, df2["delta_y_m"].values, 1)
fit_y = np.polyval(coef, df2["T2_s2"].values)
plt.plot(df2["T2_s2"], fit_y)
plt.xlabel(r"$T^2$ [s$^2$]")
plt.ylabel(r"$\Delta y$ [m]")
plt.title(f"Cross–Hall drift: slope ≈ {coef[0]:.3e} m/s$^2$")
f2b = save_fig_svg("exp2_dy_vs_T2.svg")


# ==========================================================
# EXPERIMENT 3: Rotor sidebands and holonomy shift
# ==========================================================
I_rot     = 8.4e-39
phi_theta = 0.6*np.pi
Lmax      = 6

ells = np.arange(-Lmax, Lmax+1)
E = (hbar**2/(2*I_rot)) * (ells - phi_theta/(2*np.pi))**2

plt.figure()
# stem-like plot using vlines + markers (portable across versions)
plt.vlines(ells, 0.0, E)
plt.plot(ells, E, "o")
plt.xlabel(r"$\ell$")
plt.ylabel("Energy [J]")
plt.title("Rotor sidebands and holonomy shift")
f3 = save_fig_svg("exp3_rotor_levels.svg")

df3 = pd.DataFrame({"ell": ells, "Energy_J": E})
c3 = save_csv(df3, "exp3_rotor_levels.csv")


# ==========================================================
# EXPERIMENT 4: Bounce a_min and WDW barrier coefficient
# ==========================================================
I0      = 1.0e-38
Sigma2  = (1.0e-6)**2
k_curv  = 1.0e-2
ell_wdw = 3
Pi_grid = np.geomspace(1e-40, 1e-34, 60)

A_vals  = (8*np.pi*G/3.0) * (Pi_grid**2)/(2*I0)
a_min   = ((A_vals + Sigma2)/k_curv)**0.25
C_wdw   = (ell_wdw**2 * hbar**2)/(2*I0)

plt.figure()
plt.plot(Pi_grid, a_min)
plt.xscale("log"); plt.yscale("log")
plt.xlabel(r"$\Pi_\theta$  [SI units]")
plt.ylabel(r"$a_{\min}$  [model units]")
plt.title("Bounce scale $a_{\min}$ vs. $\Pi_\theta$")
f4 = save_fig_svg("exp4_bounce_scan.svg")

df4 = pd.DataFrame({"Pi_theta": Pi_grid, "A_value": A_vals, "a_min": a_min})
c4 = save_csv(df4, "exp4_bounce_scan.csv")

print(f"WDW inverse-square barrier coefficient C = {C_wdw:.3e} J (minisuperspace units)")


# ==========================================================
# EXPERIMENT 5: Shared-range Yukawa profiles
# ==========================================================
lam   = 1.0e-3
alphaG = 1.0e-2
eps2   = 1.0e-6
r = np.geomspace(1e-5, 1e-1, 400)

yuk = np.exp(-r/lam)
frac_G   = alphaG * yuk
frac_EM  = eps2 * yuk

plt.figure()
plt.plot(r, yuk)
plt.xscale("log")
plt.xlabel("r [m]")
plt.ylabel(r"$e^{-r/\lambda_\theta}$")
plt.title("Yukawa profile vs. distance")
f5a = save_fig_svg("exp5_yukawa_profile.svg")

plt.figure()
plt.plot(r, frac_G, label="gravity fraction")
plt.plot(r, frac_EM, label="EM fraction")
plt.xscale("log")
plt.xlabel("r [m]")
plt.ylabel("fractional deviation")
plt.title("Fractional deviations vs. distance")
plt.legend()
f5b = save_fig_svg("exp5_fractional_deviation.svg")

df5 = pd.DataFrame({"r_m": r, "yukawa": yuk, "frac_gravity": frac_G, "frac_EM": frac_EM})
c5 = save_csv(df5, "exp5_yukawa_profiles.csv")


# ==========================================================
# SIMPLE CONCEPTUAL SVGs (hand-drawn via raw SVG)
# ==========================================================
def write_svg_string(name, svg_str):
    p = FIGS / name
    with open(p, "w") as f:
        f.write(svg_str)
    print(f"[SVG] {p}")
    return p

# 1) θ–AB cartoon: MZI arms, a knob (theta) and a loop showing 2π periodicity
svg_theta_ab = f'''<svg xmlns="http://www.w3.org/2000/svg" width="900" height="500" viewBox="0 0 900 500">
  <rect width="100%" height="100%" fill="white"/>
  <!-- Beam splitter and arms -->
  <line x1="120" y1="250" x2="260" y2="250" stroke="black" stroke-width="3"/>
  <line x1="260" y1="250" x2="420" y2="140" stroke="black" stroke-width="3"/>
  <line x1="260" y1="250" x2="420" y2="360" stroke="black" stroke-width="3"/>
  <line x1="420" y1="140" x2="700" y2="140" stroke="black" stroke-width="3"/>
  <line x1="420" y1="360" x2="700" y2="360" stroke="black" stroke-width="3"/>
  <!-- Recombiner -->
  <line x1="700" y1="140" x2="820" y2="250" stroke="black" stroke-width="3"/>
  <line x1="700" y1="360" x2="820" y2="250" stroke="black" stroke-width="3"/>
  <!-- Detectors -->
  <circle cx="850" cy="250" r="8" fill="black"/>
  <text x="860" y="255" font-size="18" font-family="Arial">Detector</text>
  <!-- Theta knob -->
  <circle cx="540" cy="250" r="60" fill="none" stroke="#555" stroke-width="2"/>
  <path d="M 540 190 A 60 60 0 1 1 539.9 190" fill="none" stroke="#999" stroke-width="4" stroke-dasharray="6,6"/>
  <line x1="540" y1="250" x2="585" y2="225" stroke="#cc6600" stroke-width="5"/>
  <text x="510" y="330" font-size="20" font-family="Arial" fill="#cc6600">θ gate</text>
  <!-- Holonomy loop label -->
  <path d="M 520 250 C 520 215 560 215 560 250 C 560 285 520 285 520 250" fill="none" stroke="#0066cc" stroke-width="3"/>
  <text x="565" y="255" font-size="18" font-family="Arial" fill="#0066cc">Δφ = φθ (mod 2π)</text>
  <!-- Null EM badge -->
  <rect x="80" y="60" width="200" height="40" fill="#eef" stroke="#88a"/>
  <text x="100" y="85" font-size="18" font-family="Arial" fill="#223">Null spatial EM</text>
  <!-- Caption -->
  <text x="100" y="470" font-size="18" font-family="Arial">θ–AB @ null EM: sweeping φθ by 2π returns the fringes.</text>
</svg>'''
s1 = write_svg_string("S1_theta_AB_cartoon.svg", svg_theta_ab)

# 2) Cross–Hall drift cartoon: gradient of A_theta and sideways deflection
svg_cross_hall = f'''<svg xmlns="http://www.w3.org/2000/svg" width="900" height="500" viewBox="0 0 900 500">
  <rect width="100%" height="100%" fill="white"/>
  <!-- Axes -->
  <line x1="100" y1="400" x2="800" y2="400" stroke="black" stroke-width="2"/>
  <line x1="100" y1="100" x2="100" y2="400" stroke="black" stroke-width="2"/>
  <text x="805" y="405" font-size="16" font-family="Arial">x</text>
  <text x="85" y="95"  font-size="16" font-family="Arial">y</text>
  <!-- A_theta gradient field (blue arrows up) -->
  <!-- draw several arrows along x -->
  {''.join([f'<line x1="{150+60*i}" y1="220" x2="{150+60*i}" y2="160" stroke="#2b6cb0" stroke-width="3"/>' for i in range(9)])}
  <text x="120" y="150" font-size="16" font-family="Arial" fill="#2b6cb0">∂y Aθ &gt; 0</text>
  <!-- Wavepacket trajectory -->
  <path d="M 120 360 C 260 360 360 330 480 300 C 600 270 700 240 780 220" fill="none" stroke="#cc0000" stroke-width="4"/>
  <circle cx="120" cy="360" r="6" fill="#cc0000"/>
  <text x="130" y="375" font-size="16" font-family="Arial" fill="#cc0000">packet</text>
  <!-- Sideways arrow -->
  <line x1="500" y1="300" x2="500" y2="250" stroke="#cc0000" stroke-width="4" marker-end="url(#arrow)"/>
  <!-- θ-dot gate -->
  <rect x="300" y="60" width="180" height="36" fill="#fee" stroke="#c88"/>
  <text x="310" y="84" font-size="18" font-family="Arial" fill="#833">θ̇(t) gate (on during transit)</text>
  <!-- Marker definition -->
  <defs>
    <marker id="arrow" markerWidth="10" markerHeight="7" refX="0" refY="3.5" orient="auto">
      <polygon points="0 0, 10 3.5, 0 7" fill="#cc0000" />
    </marker>
  </defs>
  <!-- Caption -->
  <text x="100" y="460" font-size="18" font-family="Arial">Cross–Hall drift: Δy ∝ (∂y Aθ) · θ̇ · T²</text>
</svg>'''
s2 = write_svg_string("S2_cross_hall_cartoon.svg", svg_cross_hall)

# 3) Rotor spectrum cartoon: parabola with discrete levels and holonomy shift
svg_rotor = f'''<svg xmlns="http://www.w3.org/2000/svg" width="900" height="500" viewBox="0 0 900 500">
  <rect width="100%" height="100%" fill="white"/>
  <!-- Parabola -->
  <path d="M 120 380 Q 450 -40 780 380" fill="none" stroke="#333" stroke-width="3"/>
  <!-- Discrete levels (dots along vertical lines) -->
  {''.join([f'<circle cx="{160+100*i}" cy="{380 - int(0.08*(i-3)*(i-3)*280)}" r="6" fill="#1a73e8"/>' for i in range(7)])}
  <!-- Holonomy shift arrow -->
  <line x1="450" y1="420" x2="500" y2="420" stroke="#d97706" stroke-width="4" marker-end="url(#arrow2)"/>
  <text x="505" y="425" font-size="18" font-family="Arial" fill="#d97706">shift by φθ / (2π)</text>
  <!-- Axis labels -->
  <text x="430" y="470" font-size="16" font-family="Arial">ℓ</text>
  <text x="95" y="220"  font-size="16" font-family="Arial" transform="rotate(-90 95,220)">Energy</text>
  <!-- Marker -->
  <defs>
    <marker id="arrow2" markerWidth="10" markerHeight="7" refX="0" refY="3.5" orient="auto">
      <polygon points="0 0, 10 3.5, 0 7" fill="#d97706" />
    </marker>
  </defs>
  <!-- Caption -->
  <text x="100" y="460" font-size="18" font-family="Arial">Rotor levels: Eℓ ∝ (ℓ − φθ/2π)² and ΔE ≈ ℏ²/(2I)</text>
</svg>'''
s3 = write_svg_string("S3_rotor_levels_cartoon.svg", svg_rotor)

# 4) Bounce schematic: effective terms vs a and a_min marker
svg_bounce = f'''<svg xmlns="http://www.w3.org/2000/svg" width="900" height="500" viewBox="0 0 900 500">
  <rect width="100%" height="100%" fill="white"/>
  <!-- Axes -->
  <line x1="120" y1="420" x2="800" y2="420" stroke="black" stroke-width="2"/>
  <line x1="120" y1="80" x2="120" y2="420" stroke="black" stroke-width="2"/>
  <text x="805" y="425" font-size="16" font-family="Arial">a</text>
  <text x="100" y="75"  font-size="16" font-family="Arial">H² (schematic)</text>
  <!-- a^-6 term (falls fast) -->
  <path d="M 140 100 C 220 140 300 200 400 300 C 520 360 640 390 780 410" fill="none" stroke="#1f77b4" stroke-width="3"/>
  <text x="420" y="290" font-size="16" font-family="Arial" fill="#1f77b4">~ (A+Σ²)/a⁶</text>
  <!-- curvature -k/a^2 term (rises toward small a, shown as baseline subtracted) -->
  <path d="M 140 410 C 260 360 420 320 620 300 C 720 295 770 295 780 295" fill="none" stroke="#2ca02c" stroke-width="3"/>
  <text x="640" y="295" font-size="16" font-family="Arial" fill="#2ca02c">-k/a²</text>
  <!-- Sum crosses zero -> a_min -->
  <line x1="500" y1="420" x2="500" y2="260" stroke="#d62728" stroke-width="2" stroke-dasharray="6,5"/>
  <circle cx="500" cy="420" r="5" fill="#d62728"/>
  <text x="480" y="450" font-size="16" font-family="Arial" fill="#d62728">a_min</text>
  <!-- Caption -->
  <text x="100" y="470" font-size="18" font-family="Arial">Bounce: H² ≈ (A+Σ²)/a⁶ − k/a² ⇒ turning point at a_min</text>
</svg>'''
s4 = write_svg_string("S4_bounce_schematic.svg", svg_bounce)

# Index CSV (maps outputs to suggested paper sections)
index_rows = [
    ("exp1_fringe_vs_phi.csv", "figs_svg/exp1_fringe_vs_phi.svg", "§2.1 θ–AB (null EM)"),
    ("exp2_drift_T2.csv", "figs_svg/exp2_dy_vs_T2.svg", "§2.2 Cross–Hall drift (scaling)"),
    ("exp2_drift_T2.csv", "figs_svg/exp2_y_traj.svg", "§2.2 Cross–Hall drift (trajectory)"),
    ("exp3_rotor_levels.csv", "figs_svg/exp3_rotor_levels.svg", "§2.3 Rotor sidebands"),
    ("exp4_bounce_scan.csv", "figs_svg/exp4_bounce_scan.svg", "§3.2 Bounce a_min vs Π_θ"),
    ("exp5_yukawa_profiles.csv", "figs_svg/exp5_yukawa_profile.svg", "§6.3 Yukawa profile"),
    ("exp5_yukawa_profiles.csv", "figs_svg/exp5_fractional_deviation.svg", "§6.3 Fractional deviations"),
    ("(n/a)", "figs_svg/S1_theta_AB_cartoon.svg", "§0/§2.1 Educational schematic"),
    ("(n/a)", "figs_svg/S2_cross_hall_cartoon.svg", "§2.2 Educational schematic"),
    ("(n/a)", "figs_svg/S3_rotor_levels_cartoon.svg", "§2.3 Educational schematic"),
    ("(n/a)", "figs_svg/S4_bounce_schematic.svg", "§3.2 Educational schematic"),
]
df_idx = pd.DataFrame(index_rows, columns=["csv_path", "svg_path", "intended_section"])
idx_csv = save_csv(df_idx, "INDEX_outputs_map.csv")

print("\\nAll done. Files are under figs_svg and data_csv.")


[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp1_fringe_vs_phi.svg
[CSV] C:\workspace\Physics\X-theta-framework\paper\data_csv\exp1_fringe_vs_phi.csv
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp2_y_traj.svg
[CSV] C:\workspace\Physics\X-theta-framework\paper\data_csv\exp2_drift_T2.csv
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp2_dy_vs_T2.svg
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp3_rotor_levels.svg
[CSV] C:\workspace\Physics\X-theta-framework\paper\data_csv\exp3_rotor_levels.csv
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp4_bounce_scan.svg
[CSV] C:\workspace\Physics\X-theta-framework\paper\data_csv\exp4_bounce_scan.csv
WDW inverse-square barrier coefficient C = 5.005e-30 J (minisuperspace units)
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp5_yukawa_profile.svg
[SVG] C:\workspace\Physics\X-theta-framework\paper\figs_svg\exp5_fractional_deviation.svg
[CSV] C:\workspace\Physi